In [ ]:
from IPython.display import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz as sv
from autoviz.AutoViz_Class import AutoViz_Class

To understand the Olist dataset, we will focus on the data that are at the core of any e-commerce organisation business model. We will focus on orders, product purchased, customers and their reviews.<br>
    - The order gives us information about who buys what and when.<br>
    - The product purchased tells us what drives the revenue, it allows us to see bestselling products, poor working product, allows category level analysis and can be connected to review, returns etc.<br>
    - The customers data allows segmentation and retention analysis<br>
    - The reviews allows us to perform sentiment analysis, show satisfaction and dissatisfaction.<br>
    <br>
The **Seller** workflow ⚙

The seller:

1. joins Olist
2. uploads their product catalogues
    - (Olist) displays these catalogues to existing marketplaces (Amazon, Bahia, Walmart, ...)
3. gets notified whenever a product is sold
4. hands over the ordered items to third-party logistic carriers

*Note: Multiple sellers can be involved in one customer's order!*

The **Customer** workflow ⚙

The customer:

1. browses products on marketplaces (Amazon, Bahia, Walmart, ...)
2. purchases products listed via store
3. gets an expected date for delivery
    - *ETA = Estimated Time of Arrival (of the orders)*
4. receives the order(s)
5. leaves a review

*Note: Between 2016 and mid-2018, a review could be left as soon as the order was sent, meaning that a customer could potentially leave a review for a product they hadn't received yet!*
It is showing the whole customer journey, from browsing to placing an order, receiving the product(s) he purchased to leaving a review. 

In [ ]:
Image("Image/olist_erd.png")

In [ ]:
# Load CSVs
orders = pd.read_csv("data/olist_orders_dataset.csv")
order_items = pd.read_csv("data/olist_order_items_dataset.csv")
order_payments = pd.read_csv("data/olist_order_payments_dataset.csv")
order_reviews = pd.read_csv("data/olist_order_reviews_dataset.csv")
customers = pd.read_csv("data/olist_customers_dataset.csv")
products = pd.read_csv("data/olist_products_dataset.csv")
sellers = pd.read_csv("data/olist_sellers_dataset.csv")
categories = pd.read_csv("data/product_category_name_translation.csv")

In [ ]:
# Merge datasets (on 'order_id' or other keys)
df = orders.merge(order_items, on='order_id', how='left') \
           .merge(order_payments, on='order_id', how='left') \
           .merge(order_reviews, on='order_id', how='left') \
           .merge(customers, on='customer_id', how='left') \
           .merge(products, on='product_id', how='left') \
           .merge(sellers, on='seller_id', how='left') \
           .merge(categories, on='product_category_name', how='left')
df.head()

### Basic EDA

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df.isna().sum()

In [ ]:
df = df.drop(columns = ["review_comment_title", "review_comment_message"]) # Too many missing values.

In [ ]:
# transform the date related data to the right format
df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"])
df["order_approved_at"] = pd.to_datetime(df["order_approved_at"])
df["order_delivered_carrier_date"] = pd.to_datetime(df["order_delivered_carrier_date"])
df["order_delivered_customer_date"] = pd.to_datetime(df["order_delivered_customer_date"])
df["order_estimated_delivery_date"] = pd.to_datetime(df["order_estimated_delivery_date"])
df["review_creation_date"] = pd.to_datetime(df["review_creation_date"])
df["review_answer_timestamp"] = pd.to_datetime(df["review_answer_timestamp"])

In [ ]:
# To enhance the understanding of customers satisfaction, we calculate if deliveries were late or early
df["order_reception_delay"] = df["order_estimated_delivery_date"] - df["order_delivered_customer_date"]

In [ ]:
df.columns

In [14]:
basic_eda_report = sv.analyze(df)

                                             |     | [  0%]   00:00 -> (? left)

In [15]:
basic_eda_report.show_html("basic_eda_report.html")

Report basic_eda_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [16]:
AV = AutoViz_Class()

In [ ]:
report = AV.AutoViz(df)

Shape of your Data Set loaded: (119143, 39)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
order_reception_delay of type=timedelta64[ns] is not classified
    Number of Numeric Columns =  15
    Number of Integer-Categorical Columns =  1
    Number of String-Categorical Columns =  11
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  0
    Number of Numeric-Boolean Columns =  0
    Number of Discrete String Columns =  1
    Number of NLP String Columns =  3
    Number of Date Time Columns =  7
    Number of ID Columns =  0
    Number of Columns to Delete =  0
No of columns classified 38 does not match 39 total cols. Continuing...
 Missing columns = ['order_reception_delay']
        No variables removed

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
order_id,object,0.000000,83,,,No issue
customer_id,object,0.000000,83,,,No issue
order_status,object,0.000000,0,,,"6 rare categories: ['canceled', 'unavailable', 'invoiced', 'processing', 'created', 'approved']. Group them into a single category or drop the categories."
order_purchase_timestamp,datetime64[ns],0.000000,82,,,Possible date-time colum: transform before modeling step.
order_approved_at,datetime64[ns],0.148561,76,,,"Possible date-time colum: transform before modeling step., 177 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types:"
order_delivered_carrier_date,datetime64[ns],1.750837,68,,,"Possible date-time colum: transform before modeling step., 2086 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types:"
order_delivered_customer_date,datetime64[ns],2.871339,80,,,"Possible date-time colum: transform before modeling step., 3421 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types:"
order_estimated_delivery_date,datetime64[ns],0.000000,0,,,Possible date-time colum: transform before modeling step.
order_item_id,float64,0.699160,NA,1.000000,21.000000,"833 missing values. Impute them with mean, median, mode, or a constant value such as 123., Column has 14665 outliers greater than upper bound (1.00) or lower than lower bound(1.00). Cap them or remove them."
product_id,object,0.699160,27,,,"833 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"


Number of All Scatter Plots = 120
